In [5]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [11]:
data=pd.read_csv("C:/Users/user/Python self-study/MOB 스터디/전송파일/fac/fac_final_data2.csv", engine="python", encoding = "euc-kr")

In [18]:
data.shape

(1728, 15)

In [24]:
#이전 submission데이터가 없을때
train_data=pd.read_csv("C:/Users/user/Python self-study/MOB 스터디/전송파일/fac/fac_final_data.csv",
                       engine="python", encoding="utf-8")
train_data=train_data.drop("Unnamed: 0",axis=1)
train_data.head()

sub_set=train_data[-10:]
sub_set_y=sub_set["사용량(kWh)"]
sub_set_x=sub_set.drop("사용량(kWh)",axis=1)
sub_set.head()

,place,date,평균기온(C),최고기온(C),최저기온(C),평균상대습도(%),월합강수량(00~24h만)(mm),평균풍속(m/s),평균운량,고객호수,사용량(kWh),전기요금(원),평균판매단가(원/kWh),증감률 (%),생산지수(2015=100)
1782,15,7,26.3,35.9,16.5,70.0,185.8,1.5,NaN,21368,3488451079,397291826762,113.9,1.8,128.3
1783,15,8,27.2,37.8,17.0,66.0,282.7,1.7,5.9,21432,3482154319,397643678457,114.2,5.6,129.2
1784,15,9,19.3,28.8,7.4,70.0,124.6,1.6,5.4,21505,3251631341,293284703022,90.2,-6.6,120.5
1785,15,10,11.1,24.3,-1.4,70.0,99.8,1.3,4.2,21561,3371245509,300229566920,89.1,3.5,128.5
1786,15,11,6.2,19.3,-5.5,68.0,48.3,0.9,3.9,21585,3342937344,378228369292,113.1,-4.2,124.4


In [25]:
#실제 데이터
train_data=pd.read_csv("C:/Users/user/Python self-study/MOB 스터디/전송파일/fac/fac_final_data2.csv",
                       engine="python", encoding="euc-kr")
train_data=train_data.drop("Unnamed: 0",axis=1)
train_data.head()
submission_data=pd.read_csv("C:/Users/user/Python self-study/MOB 스터디/전송파일/fac/submission.csv",
                       engine="python", encoding="euc-kr")
submission=submission_data.drop("Unnamed: 0",axis=1)
submission_y=submission["사용량(kWh)"].values
submission=submission.drop("사용량(kWh)",axis=1)

In [26]:
#카글 lightgbm
#lightgbm모델은 마지막에 하자
#1. 지역정보는 배제할때
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import time

#
# Prepare the data
#
train=train_data
print(train.columns)

# get the labels
y = train["사용량(kWh)"].values
train=train.drop('사용량(kWh)', axis=1)
x = train.values
print("x: ",x.shape)
print("y: ",y.shape)
#
# Create training and validation sets
#
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42)#, stratify=y)

#
# Create the LightGBM data containers
#
categorical_features = [c for c, col in enumerate(train.columns) if ('place' in col) or (('date' in col))]
#print(categorical_features)
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test, categorical_feature=categorical_features)


#
# Train the model
#

parameters = {
    #'application': 'binary',
    'task':'predict',
    'objective': 'regression',
    'metric': 'mae', #or 'l1'
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 600,
    #'min_data':400,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.02,
    'verbose': 0,
    'max_depth':200
}
start=time.time()
model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=10000,
                       early_stopping_rounds=100)
print("done!!!")
#
# Create a submission
#

#submission = pd.read_csv('test_data/test.csv',engine='c', encoding='utf-8')
#ids = submission['id'].values
#submission.drop('id', inplace=True, axis=1)

#submission=sub_set_x
x = submission.values
y = model.predict(x)
print("time: ",time.time() - start,"sec")
#output = pd.DataFrame({'id': ids, 'target': y})
#output.to_csv("submission.csv", index=False)

Index(['place', 'date', '평균기온(C)', '최고기온(C)', '최저기온(C)', '평균상대습도(%)',
       '월합강수량(00~24h만)(mm)', '평균풍속(m/s)', '평균운량', '고객호수', '사용량(kWh)',
       '전기요금(원)', '평균판매단가(원/kWh)', '증감률 (%)', '생산지수(2015=100)'],
      dtype='object')
x:  (1728, 14)
y:  (1728,)
[1]	valid_0's l1: 9.96677e+08
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 9.77628e+08
[3]	valid_0's l1: 9.63156e+08
[4]	valid_0's l1: 9.44602e+08
[5]	valid_0's l1: 9.26284e+08
[6]	valid_0's l1: 9.09166e+08
[7]	valid_0's l1: 8.91536e+08
[8]	valid_0's l1: 8.78982e+08
[9]	valid_0's l1: 8.61929e+08
[10]	valid_0's l1: 8.49504e+08
[11]	valid_0's l1: 8.33593e+08
[12]	valid_0's l1: 8.17772e+08
[13]	valid_0's l1: 8.03072e+08
[14]	valid_0's l1: 7.87876e+08
[15]	valid_0's l1: 7.7258e+08
[16]	valid_0's l1: 7.5814e+08
[17]	valid_0's l1: 7.44424e+08
[18]	valid_0's l1: 7.42012e+08
[19]	valid_0's l1: 7.28127e+08
[20]	valid_0's l1: 7.14043e+08
[21]	valid_0's l1: 7.00648e+08
[22]	valid_0's l1: 6.87593e+08
[23]	valid_0'

C:\Users\user\Anaconda3\lib\site-packages\lightgbm\basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[42]	valid_0's l1: 4.74663e+08
[43]	valid_0's l1: 4.66182e+08
[44]	valid_0's l1: 4.65141e+08
[45]	valid_0's l1: 4.56962e+08
[46]	valid_0's l1: 4.48709e+08
[47]	valid_0's l1: 4.41074e+08
[48]	valid_0's l1: 4.34897e+08
[49]	valid_0's l1: 4.2732e+08
[50]	valid_0's l1: 4.1977e+08
[51]	valid_0's l1: 4.12505e+08
[52]	valid_0's l1: 4.05166e+08
[53]	valid_0's l1: 3.98015e+08
[54]	valid_0's l1: 3.91016e+08
[55]	valid_0's l1: 3.84239e+08
[56]	valid_0's l1: 3.77496e+08
[57]	valid_0's l1: 3.7656e+08
[58]	valid_0's l1: 3.69935e+08
[59]	valid_0's l1: 3.63537e+08
[60]	valid_0's l1: 3.57305e+08
[61]	valid_0's l1: 3.50923e+08
[62]	valid_0's l1: 3.44694e+08
[63]	valid_0's l1: 3.38834e+08
[64]	valid_0's l1: 3.32937e+08
[65]	valid_0's l1: 3.27352e+08
[66]	valid_0's l1: 3.21698e+08
[67]	valid_0's l1: 3.16343e+08
[68]	valid_0's l1: 3.12624e+08
[69]	valid_0's l1: 3.07524e+08
[70]	valid_0's l1: 3.02452e+08
[71]	valid_0's l1: 2.98868e+08
[72]	valid_0's l1: 2.93993e+08
[73]	valid_0's l1: 2.89253e+08
[74]	valid_

In [27]:
#정획도
pred_=[]
for i in y:
    pred_.append(int(i))
correct_=np.array(submission_y)
pred_=np.array(pred_)
#rmse

print("예측값")
print(pred_[:10])
print("정답")
print(correct_[:10])
#from sklearn.metrics import mean_squared_error
#RMSE= mean_squared_error(correct_, pred_)**0.5
#print("RMSE: ",RMSE)
accuracy=np.fabs(pred_-correct_)
accuracy=accuracy/pred_
accuracy=(1-accuracy)*100
print("accuracy: ",format(np.mean(accuracy),".3f"),"%")

예측값
[ 494550778  414198704  378283342  379068894 1213247394 1006465686
 1108402758 1108508381  639483905  536968333]
정답
[ 464109345  442186825  399569458  390954203 1224965166 1008592370
 1121224114 1103065852  582592914  499178837]
accuracy:  95.059 %
